## IMPORTAR LAS LIBRERIAS

In [24]:
import numpy as np
import pandas as pd
import cloudpickle

from janitor import clean_names

from sklearn.preprocessing import MinMaxScaler

from lightgbm import LGBMClassifier

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

## CARGAR LOS DATOS

### Ruta del proyecto

In [25]:
ruta_proyecto = 'C:/Users/Alfonso/OneDrive/Documentos_/Formación/Bootcamp_DS/Repo_DS/Proyect_Break_ML/00_PROYECTO_ML'

### Nombre del fichero de datos

In [26]:
nombre_fichero_datos = 'marketing_campaign.xlsx'

### Cargar los datos

In [27]:
ruta_completa = ruta_proyecto + '/02_Datos/01_Originales/' + nombre_fichero_datos

df = pd.read_excel(ruta_completa,index_col=0)

### Selección de las variables finales

#### Lista de variables finales

In [28]:
nombre_variables_finales = ruta_proyecto + '/05_Resultados/' + 'variables_finales.pickle'

pd.read_pickle(nombre_variables_finales).sort_index().index.to_list()

['household_members_mms',
 'income_mms',
 'mntgoldprods_mms',
 'mntmeatproducts_mms',
 'mntwines_mms',
 'numcatalogpurchases_mms',
 'numstorepurchases_mms',
 'numwebvisitsmonth_mms',
 'total_cmp_mms']

In [29]:
variables_finales = ['household_members',
 'income',
 'mntgoldprods',
 'mntmeatproducts',
 'mntwines',
 'numcatalogpurchases',
 'numstorepurchases',
 'numwebvisitsmonth',
 'total_cmp' ]

## ESTRUCTURA DE LOS DATASETS

### Corregir los nombres

In [30]:
df = clean_names(df)

### Eliminar registros

#### Por duplicados

In [31]:
df.drop_duplicates(inplace = True)

#### Por Nulos

### Creación nuevas variables

In [33]:
# Miembros totales en casa
dicc_status = {
    'Together': 2,  # Generalmente representa a dos personas viviendo juntas
    'Divorced': 1,  # Generalmente representa a una persona que está divorciada
    'Married': 2,   # Generalmente representa a dos personas casadas
    'Single': 1,    # Generalmente representa a una persona soltera
    'Widow':1,      # Generalmente representa a una persona viuda
    'OTROS': 1      # 1 por defecto    
}

df['status_members'] = df['marital_status'].map(dicc_status)
df['household_members'] = df['status_members']+df['kidhome']+df['teenhome']

# Nº ofertas aceptadas
df['total_cmp'] = df.filter(like='accepted').apply(pd.to_numeric, errors='coerce').sum(axis=1)

# Elimino columnas intermedias
df = df.drop(columns=['status_members'])

#### Para x

Quedarse solo con las de la lista.

In [34]:
x = df[variables_finales].copy()

#### Para y

Especificar la target.

In [36]:
target = 'response'

Crear el y.

In [37]:
y = df[target].copy()

## CREAR EL PIPELINE

### Instanciar el tratamiento de outliers

In [40]:
### Tratamiento de outliers
# Winsorización automática para cada variable
def winsorizacion(df, var_winsorizar, p_min=0.01, p_max=0.99):
    df = df.copy()
    for variable in var_winsorizar:
        lower_quantile = df[variable].quantile(p_min)
        upper_quantile = df[variable].quantile(p_max)
        if pd.api.types.is_integer_dtype(df[variable]):
            lower_quantile = int(round(lower_quantile))
            upper_quantile = int(round(upper_quantile))
        df[variable] = df[variable].clip(lower=lower_quantile, upper=upper_quantile)
    return df
var_winsorizar = ['income',
 'mntgoldprods',
 'mntmeatproducts',
 'mntwines',
 'numcatalogpurchases',
 'numstorepurchases',
 'numwebvisitsmonth']

win = FunctionTransformer(winsorizacion, kw_args={'var_winsorizar': var_winsorizar, 'p_min':0.01,'p_max':0.996})

### Instanciar transformación de variables

In [41]:
var_mms = ['household_members',
 'income',
 'mntgoldprods',
 'mntmeatproducts',
 'mntwines',
 'numcatalogpurchases',
 'numstorepurchases',
 'numwebvisitsmonth',
 'total_cmp']
mms = MinMaxScaler()

### Crear el pipe del preprocesamiento

#### Crear el column transformer

In [42]:
ct = make_column_transformer(
    (win, var_winsorizar),
    (mms, var_mms),
    remainder='drop')

#### Crear el pipeline del preprocesamiento

In [43]:
pipe_prepro = make_pipeline(ct)

### Instanciar el modelo

#### Instanciar el algoritmo

In [44]:
modelo = LGBMClassifier(n_jobs = -1, 
                       verbosity = -1,
                       is_unbalance= True,
                        learning_rate= 0.06,
                        max_depth= 2,
                        min_child_samples= 2,
                        min_child_weight= 28,
                        n_estimators= 95,
                        objective= 'binary' )

#### Crear el pipe final de entrenamiento

In [45]:
pipe_entrenamiento = make_pipeline(pipe_prepro,modelo)

#### Guardar el pipe final de entrenamiento

In [46]:
nombre_pipe_entrenamiento = 'pipe_entrenamiento.pickle'

ruta_pipe_entrenamiento = ruta_proyecto + '/04_Modelos/' + nombre_pipe_entrenamiento

with open(ruta_pipe_entrenamiento, mode='wb') as file:
   cloudpickle.dump(pipe_entrenamiento, file)

#### Entrenar el pipe final de ejecución

In [47]:
pipe_ejecucion = pipe_entrenamiento.fit(x,y)

## GUARDAR EL PIPE

### Nombre del pipe final de ejecución

In [48]:
nombre_pipe_ejecucion = 'pipe_ejecucion.pickle'

### Guardar el pipe final de ejecución

In [49]:
ruta_pipe_ejecucion = ruta_proyecto + '/04_Modelos/' + nombre_pipe_ejecucion

with open(ruta_pipe_ejecucion, mode='wb') as file:
   cloudpickle.dump(pipe_ejecucion, file)